In [21]:
from process_data.load_data import load_data, serialize_Result_JSON
from process_data.get_result import get_result
import shubhlipi as sh
import os
import jinja2
import dotenv
from process_data import MainDataType, ResultDataType

# Not using pandas as it is not suitabke to run in a microservice


In [22]:
# Gettting Result and `data``
data = load_data(
    sh.read("./data/answer_key.csv"), sh.read("./data/question_paper_html")
)
result = get_result(data)


In [23]:
# functions to hydrate the markdown data


def render_jinja_temp(fl: str, **o):
    return (
        jinja2.Environment(loader=jinja2.FileSystemLoader("./template/"))
        .get_template(fl)
        .render(**o)
    )


def get_result_markdown(data: MainDataType, result: ResultDataType) -> str:
    # We can also pass the data as a dict or even directly as shown below
    return render_jinja_temp("result_temp.md.j2", data=data, result=result)


def get_questions_markdown(data: MainDataType, sc: ResultDataType) -> str:
    return render_jinja_temp("questions_temp.md.j2", data=data.__dict__)


In [24]:
# Deletiing the result folder
if os.path.exists("./result"):
    sh.delete_folder("./result")
sh.makedir("./result")


# Saving the result
sh.write("./result/result.md", get_result_markdown(data, result))
# Saving the questions
sh.write("./result/questions.md", get_questions_markdown(data, result))


In [25]:
# Loading env variables
if os.path.isfile("../py/.env"):
    dotenv.load_dotenv("../py/.env")

SAVE_RESULT_TO_DATABASE = False
# The above will be true if we want to save the result and the data into the database
# This example data will be used a sample result for the frontend
if SAVE_RESULT_TO_DATABASE:
    print("Saving to database")
    from deta import Deta

    deta = Deta(sh.from_base64(os.getenv("DETA_KEY")))

    deta.Base("info").put(
        {"name": "", "DOB": "", "date": "", "time": "", "roll": ""}, "sample_result"
    )

    deta.Base("data").put(
        {
            "result": serialize_Result_JSON(result),
            "data": data.__dict__,
        },
        "sample_result",
    )
    print("Saved to database")
